# Data Analytics for Customer Table

This is a Notebook used for submission related to Analytics and Insights from the Data

## 1. Library and Data Import

In [2]:
import pandas as pd
import numpy as np
from typing import Dict, Any, List, Union
import matplotlib.pyplot as plt
import random
import plotly.express as px
import plotly.graph_objects as go
from typing import List
import calendar
from datetime import timedelta

In [ ]:
# Import and Load Data using Pandas
customer_data = pd.read_csv('xx/customers-100000.csv')
customer_data.head(5)

,Index,Customer Id,First Name,Last Name,Company,City,Country,Phone 1,Phone 2,Email,Subscription Date,Website
0,1,ffeCAb7AbcB0f07,Jared,Jarvis,Sanchez-Fletcher,Hatfieldshire,Eritrea,274.188.8773x41185,001-215-760-4642x969,gabriellehartman@benjamin.com,2021-11-11,https://www.mccarthy.info/
1,2,b687FfC4F1600eC,Marie,Malone,Mckay PLC,Robertsonburgh,Botswana,283-236-9529,(189)129-8356x63741,kstafford@sexton.com,2021-05-14,http://www.reynolds.com/
2,3,9FF9ACbc69dcF9c,Elijah,Barrera,Marks and Sons,Kimbury,Barbados,8252703789,459-916-7241x0909,jeanettecross@brown.com,2021-03-17,https://neal.com/
3,4,b49edDB1295FF6E,Sheryl,Montgomery,"Kirby, Vaughn and Sanders",Briannaview,Antarctica (the territory South of 60 deg S),425.475.3586,(392)819-9063,thomassierra@barrett.com,2020-09-23,https://www.powell-bryan.com/
4,5,3dcCbFEB17CCf2E,Jeremy,Houston,Lester-Manning,South Brianna,Micronesia,+1-223-666-5313x4530,252-488-3850x692,rubenwatkins@jacobs-wallace.info,2020-09-18,https://www.carrillo.com/


## 2. Data Exploration and Data Cleansing

### 2.1 Standardizing Data
This process is used to standardize the format of all data by deleting white space and lowering all characters

In [4]:
def data_processing (df: pd.DataFrame) -> pd.DataFrame:
    # Convert 'subscription_date' to datetime
    working_df = df.copy()
    working_df['First Name'] = working_df['First Name'].fillna('').astype(str).str.lower().str.strip()
    working_df['Last Name'] = working_df['Last Name'].fillna('').astype(str).str.lower().str.strip()
    working_df['Email'] = working_df['Email'].astype(str).str.lower().str.strip()
    working_df['Full Name'] = (working_df['First Name'] + ' ' + working_df['Last Name']).str.strip()
    working_df['Company'] = working_df['Company'].fillna('').astype(str).str.lower().str.strip()
    working_df['Website'] = working_df['Website'].fillna('').astype(str).str.lower().str.strip()
    working_df['Country'] = working_df['Country'].fillna('').astype(str).str.lower().str.strip()
    working_df['City'] = working_df['City'].fillna('').astype(str).str.lower().str.strip()
    working_df['Subscription Date'] = pd.to_datetime(working_df['Subscription Date'])
    return working_df

customer_data = data_processing(customer_data)

In [5]:
customer_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   Index              100000 non-null  int64         
 1   Customer Id        100000 non-null  object        
 2   First Name         100000 non-null  object        
 3   Last Name          100000 non-null  object        
 4   Company            100000 non-null  object        
 5   City               100000 non-null  object        
 6   Country            100000 non-null  object        
 7   Phone 1            100000 non-null  object        
 8   Phone 2            100000 non-null  object        
 9   Email              100000 non-null  object        
 10  Subscription Date  100000 non-null  datetime64[ns]
 11  Website            100000 non-null  object        
 12  Full Name          100000 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(11)
me

In [6]:
display(customer_data.head(5))

,Index,Customer Id,First Name,Last Name,Company,City,Country,Phone 1,Phone 2,Email,Subscription Date,Website,Full Name
0,1,ffeCAb7AbcB0f07,jared,jarvis,sanchez-fletcher,hatfieldshire,eritrea,274.188.8773x41185,001-215-760-4642x969,gabriellehartman@benjamin.com,2021-11-11,https://www.mccarthy.info/,jared jarvis
1,2,b687FfC4F1600eC,marie,malone,mckay plc,robertsonburgh,botswana,283-236-9529,(189)129-8356x63741,kstafford@sexton.com,2021-05-14,http://www.reynolds.com/,marie malone
2,3,9FF9ACbc69dcF9c,elijah,barrera,marks and sons,kimbury,barbados,8252703789,459-916-7241x0909,jeanettecross@brown.com,2021-03-17,https://neal.com/,elijah barrera
3,4,b49edDB1295FF6E,sheryl,montgomery,"kirby, vaughn and sanders",briannaview,antarctica (the territory south of 60 deg s),425.475.3586,(392)819-9063,thomassierra@barrett.com,2020-09-23,https://www.powell-bryan.com/,sheryl montgomery
4,5,3dcCbFEB17CCf2E,jeremy,houston,lester-manning,south brianna,micronesia,+1-223-666-5313x4530,252-488-3850x692,rubenwatkins@jacobs-wallace.info,2020-09-18,https://www.carrillo.com/,jeremy houston


### 2.2 Checking Missing Value and Calculating NaN Rate

In [7]:
class DataProfiler:

    def __init__(self, df: pd.DataFrame):
        """
        Initializes the DataProfiler with a pandas DataFrame.

        Args:
            df: The pandas DataFrame to be profiled.
        """
        if not isinstance(df, pd.DataFrame):
            raise TypeError("Input must be a pandas DataFrame.")
        self.df = df
        print(f"DataProfiler initialized with a DataFrame of shape {df.shape}")

    def check_missing_values(self) -> pd.DataFrame:
        # Access the DataFrame stored in the instance
        missing_values = self.df.isnull().sum()
        missing_percentage = (missing_values / len(self.df)) * 100

        # Create the result DataFrame
        missing_data = pd.DataFrame({
            'Total Missing Values': missing_values,
            'Percentage (%)': missing_percentage
        })

        return missing_data

    def get_top_unique_values(self, top_n: int = 10) -> Dict[str, List[Dict[str, Union[str, int]]]]:
        """"Use this functions to calculate top unique values to check whether there is null or not"""
        results: Dict[str, List[Dict[str, Union[str, int]]]] = {}

        for col in self.df.columns:
            # Calculate value counts, including NaN values (dropna=False)
            counts_series = self.df[col].value_counts(dropna=False).head(top_n)

            # Format the result in a list of dictionaries
            column_results = []
            for value, count in counts_series.items():
                # Convert value to string and handle NaN values for consistent output
                display_value = 'None/NaN' if pd.isna(value) else str(value)

                column_results.append({
                    "value": display_value,
                    "count": int(count)
                })
            results[col] = column_results

        return results

profiler = DataProfiler(customer_data)
print("\n--- Missing Values Report ---")
missing_values_report = profiler.check_missing_values()
display(missing_values_report)


print("\n--- Top Unique Values Report (Top 10) ---")
top_unique_values_report = profiler.get_top_unique_values(top_n=10)
display(top_unique_values_report)

DataProfiler initialized with a DataFrame of shape (100000, 13)

--- Missing Values Report ---


,Total Missing Values,Percentage (%)
Index,0,0.0
Customer Id,0,0.0
First Name,0,0.0
Last Name,0,0.0
Company,0,0.0
City,0,0.0
Country,0,0.0
Phone 1,0,0.0
Phone 2,0,0.0
Email,0,0.0



--- Top Unique Values Report (Top 10) ---


{'Index': [{'value': '1', 'count': 1},
  {'value': '66651', 'count': 1},
  {'value': '66673', 'count': 1},
  {'value': '66672', 'count': 1},
  {'value': '66671', 'count': 1},
  {'value': '66670', 'count': 1},
  {'value': '66669', 'count': 1},
  {'value': '66668', 'count': 1},
  {'value': '66667', 'count': 1},
  {'value': '66666', 'count': 1}],
 'Customer Id': [{'value': 'ffeCAb7AbcB0f07', 'count': 1},
  {'value': '434D18E3801A4eF', 'count': 1},
  {'value': 'E30DEC6235F0C6a', 'count': 1},
  {'value': 'A9Ab9FCCb6c1b13', 'count': 1},
  {'value': '9f37fc720ae9A4f', 'count': 1},
  {'value': 'b8bCDcaAE7968f5', 'count': 1},
  {'value': '8692c0DBDdded6b', 'count': 1},
  {'value': 'cddE870b31c9DC6', 'count': 1},
  {'value': 'bD02651B0aCfcc0', 'count': 1},
  {'value': 'cE0bb6a6eBA72eF', 'count': 1}],
 'First Name': [{'value': 'joan', 'count': 183},
  {'value': 'audrey', 'count': 182},
  {'value': 'bridget', 'count': 182},
  {'value': 'anne', 'count': 180},
  {'value': 'melinda', 'count': 177},
 

**Summary** : Based on the analysis, no NaN values were detected in any column. As an additional validation step, we also examined the top value counts for each field. This approach helps identify hidden or improperly formatted missing values, which typically appear frequently in the dataset due to their consistent representation.

### 2.3 Data Value Analysis

There are several assumptions and conditions that need to be validated:

1. Relationships between data fields
We need to assess whether fields such as Customer ID, Customer Name, Email, and Website share similar levels of uniqueness. This helps us understand how closely these attributes are correlated and whether they represent the same entity consistently.

2. Relationship between Company Name and other attributes
We will evaluate how Company Name is associated with the customer-level information to ensure data consistency across organizations.

3. Rationale:
Preliminary inspection suggests that some email addresses do not appear to correspond to the associated customer names — indicating potential randomness or invalid entries. Therefore, it is important to confirm the uniqueness and integrity of these fields to ensure the quality of our analysis.

In [8]:
# Mapping Analysis in the level of Company Name
def company_name_website(df):
    def get_unique_values_as_list(series: pd.Series) -> List[str]:
        """Helper function to convert unique series values to a list of strings."""
        # Convert to set for uniqueness, then back to list, handling NaNs
        return list(set(series.dropna().astype(str).tolist()))
    working_df = df.copy() 
    
    # Check and Identify required columns
    required_cols = ['Company', 'Website', 'First Name', 'Last Name', 'Email']
    for col in required_cols:
        if col not in working_df.columns:
            print(f"Error: Required column '{col}' not found in DataFrame for Company/Website analysis.")
            return pd.DataFrame(), pd.DataFrame()
            
    customer_id_col = next((col for col in ['Customer_ID', 'Customer Id'] if col in working_df.columns), None)
    if not customer_id_col:
        print("Error: DataFrame must contain a 'Customer_ID' or 'Customer Id' column for aggregation.")
        return pd.DataFrame(), pd.DataFrame()


    # 1. Robust Standardization and Name Creation
    print("\n-> Starting Company Profile Analysis:")
    print("-> Standardizing Analysis Process of the Data.")

    
    # 2. Group by Company and aggregate all metrics
    company_website_report = working_df.groupby('Company').agg(
        Websites=('Website', get_unique_values_as_list),
        Customer_IDs=(customer_id_col, get_unique_values_as_list),
        Emails=('Email', get_unique_values_as_list),
        Names=('Full Name', get_unique_values_as_list)
    ).reset_index()
    
    # 3. Calculate all count metrics
    company_website_report['unique_website_count'] = company_website_report['Websites'].apply(len)
    company_website_report['unique_customer_id_count'] = company_website_report['Customer_IDs'].apply(len)
    company_website_report['unique_email_count'] = company_website_report['Emails'].apply(len)
    company_website_report['unique_name_count'] = company_website_report['Names'].apply(len)
    
    # 4. Filter and return the two reports (Issue criteria still based on Website inconsistency)
    
    # Issue criteria: Flag if the Company has > 1 unique website
    issue_criteria = (company_website_report['unique_website_count'] > 1) 

    final_report_morethan_one = company_website_report[issue_criteria]
    
    # Clean criteria: Exactly one unique website
    clean_criteria = (company_website_report['unique_website_count'] == 1)
                     
    final_report_one = company_website_report[clean_criteria]
    
    # Print summary
    if final_report_morethan_one.empty:
        print("\nAnalysis Complete: No Company/Website quality issues found (All Companies have 1 unique website).")
        print(f"   Clean companies found: {len(final_report_one)}")
    else:
        print(f"\nAnalysis Complete:\n1. Found {len(final_report_morethan_one)} Companies with multiple unique websites (identifier issue).")
        print(f"2. Found {len(final_report_one)} Companies associated with exactly one unique website.")
        
    # Sort and return the two DataFrames
    return (
        final_report_morethan_one.sort_values(by='Company').reset_index(drop=True), 
        final_report_one.sort_values(by='Company').reset_index(drop=True)
    )


company_website_issues, company_one = company_name_website(customer_data)
display(company_website_issues.head(10))
display(company_one.head(10))


-> Starting Company Profile Analysis:
-> Standardizing Analysis Process of the Data.

Analysis Complete:
1. Found 6380 Companies with multiple unique websites (identifier issue).
2. Found 65614 Companies associated with exactly one unique website.


,Company,Websites,Customer_IDs,Emails,Names,unique_website_count,unique_customer_id_count,unique_email_count,unique_name_count
0,abbott and sons,"[https://www.moss.net/, https://www.barnett.co...","[83141475D04a7d4, fbCe0abe0a52AaB, 20CF89A6239...","[tylercarly@burgess.com, hubbardjesse@payne-ma...","[eric parrish, sylvia ali, marvin bolton, colt...",5,5,5,5
1,abbott group,"[https://gilbert-mosley.com/, http://abbott-co...","[aa2Bc8ffa1CCdFF, 5cEe43E5caF31Ac, 9b5c7EaA7CF...","[mckenziebridges@whitaker.com, kerrigolden@cor...","[marvin welch, tracy potts, stanley dominguez,...",7,7,7,7
2,abbott inc,"[http://www.knapp.com/, http://booth-phelps.com/]","[A14521ae3cdDA1A, 6BdeCD1Ab54C31c]","[darinmcknight@rubio.com, prestonbryant@small-...","[julie herman, javier butler]",2,2,2,2
3,abbott llc,"[http://www.gonzales.com/, https://christian-p...","[ec2ee6BCEa932A4, d7EB6cB1B6485cA, B11A3fC9Fa9...","[ayalajasmin@wilcox.com, obowers@booth.com, br...","[riley benson, mitchell porter, devin sellers,...",7,7,7,7
4,abbott ltd,"[http://case.com/, https://www.kirby.com/, htt...","[cBBB1ebB657F46e, 7ceB44BdC61f80a, 69e4686F5b1...","[warren18@clay-collier.info, doylevalerie@roac...","[nina macdonald, whitney aguilar, wesley cummi...",4,4,4,4
5,abbott plc,"[http://mcdaniel.com/, http://klein-mathis.net...","[EeBBbB9EFc59991, BDcDaa0a1D53f23, 7C1FA801c76...","[krystaldixon@hood-villanueva.com, bob58@mccon...","[bob bradford, willie page, jay schroeder]",3,3,3,3
6,acevedo and sons,"[http://www.fry.info/, http://www.myers.com/, ...","[A8c638e8CcaBB3a, FEBDBdb478eaABF, Cb2410e56aB...","[davisivan@lam-romero.com, garcialawrence@owen...","[jimmy walker, marcus chang, bailey christian,...",8,8,8,8
7,acevedo group,"[http://www.valencia-bartlett.biz/, http://www...","[DdbEc0653fCAecF, 0ace2C8287F77AF]","[edickerson@steele-osborn.com, bowmanjermaine@...","[luke watkins, grace gould]",2,2,2,2
8,acevedo inc,"[http://mullins-combs.info/, https://barr-bren...","[851bDbbA3AFF054, f2efec1CeE0b9Ee, A6F6dEf5D2f...","[zmeza@newton-dickson.com, rickey75@holland.in...","[kelly tyler, isaac robbins, marie roth, spenc...",8,8,8,8
9,acevedo llc,"[https://mayo-swanson.com/, https://villanueva...","[f3afDe44a5cBd4a, aFBDC8CF4C7156c, E82CA8F217d...","[howardregina@mcdowell-cole.com, bridgesdougla...","[mason little, franklin adams, herbert villanu...",3,3,3,3


,Company,Websites,Customer_IDs,Emails,Names,unique_website_count,unique_customer_id_count,unique_email_count,unique_name_count
0,"abbott, bartlett and fox",[http://rojas-cantu.net/],[DC538DB4ccAbcc1],[tonyarroyo@gay.com],[zachary hall],1,1,1,1
1,"abbott, bright and barajas",[https://www.black-mcpherson.info/],[15dc84DAbaD082e],[carla79@jarvis.com],[pam christensen],1,1,1,1
2,"abbott, cannon and patterson",[https://www.wright.com/],[dF4f123fBAFe9df],[kirk85@meadows-bond.com],[adam blevins],1,1,1,1
3,"abbott, christensen and compton",[https://www.hicks-kramer.com/],[7D30B07A832d4d9],[gcameron@dixon-dougherty.info],[susan reeves],1,1,1,1
4,"abbott, cisneros and haney",[https://www.hodge-hardy.biz/],[DB3BccB6b0cd732],[robert30@mata.com],[micheal morton],1,1,1,1
5,"abbott, cohen and pittman",[https://www.rush.com/],[63Ba6B0eA01047F],[lynchpreston@ross.com],[deanna farrell],1,1,1,1
6,"abbott, collins and andersen",[http://cameron-sutton.biz/],[4c538EbbcAcCF8E],[gmcdaniel@kemp.com],[allison copeland],1,1,1,1
7,"abbott, crosby and shaw",[https://www.haynes-arellano.com/],[F3a7b01284aCfE6],[dalvarado@carson.net],[denise young],1,1,1,1
8,"abbott, escobar and allen",[http://mclaughlin.biz/],[BEA18FCF455EfBE],[hlynn@patton.com],[alicia mahoney],1,1,1,1
9,"abbott, gentry and owens",[http://foley.com/],[26Cebf7f9Db4a5E],[ohickman@martin.com],[paige farmer],1,1,1,1


In [9]:
# Mapping analysis in the level of personal name with email and website
def analyze_name_email_correlation(df: pd.DataFrame) -> pd.DataFrame:
    working_df = df.copy()
    # Define functions to collect unique values into a list
    def get_unique_values_as_list(series):
        # Convert to set for uniqueness, then back to list, handling NaNs
        return list(set(series.dropna().astype(str).tolist()))

    # 2. Group by Customer ID AND Full Name and aggregate unique Emails
    # This checks for inconsistent emails within a specific customer/name pair.
    name_email_report = working_df.groupby(['Customer Id', 'Full Name']).agg(
        Emails=('Email', get_unique_values_as_list), Website=('Website', get_unique_values_as_list)
    ).reset_index()
    
    # Rename columns to match the requested output format
    name_email_report.rename(columns={'Full Name': 'Name'}, inplace=True)
    
    # 3. Calculate the count of unique Emails
    name_email_report['unique_email_count'] = name_email_report['Emails'].apply(len)
    name_email_report['unique_website_count'] = name_email_report['Website'].apply(len)

    
    # 4. Filter and return the two reports
    
    # Issue criteria: Flag if the specific (Customer ID, Name) pair has > 1 unique email
    issue_criteria = (name_email_report['unique_email_count'] > 1) 

    final_report_morethan_one = name_email_report[issue_criteria]
    
    # Clean criteria: Exactly one unique email
    clean_criteria = (name_email_report['unique_email_count'] == 1)
                     
    final_report_one = name_email_report[clean_criteria]
    
    # Print summary
    if final_report_morethan_one.empty:
        print("\nAnalysis Complete: No data quality issues found (All Customer ID/Name pairs have 1 unique email).")
        print(f"2. Found {len(final_report_one)} unique Customer ID/Name pairs associated with exactly one email.")
    else:
        print(f"\nAnalysis Complete: 1. Found {len(final_report_morethan_one)} Customer ID/Name pairs with multiple emails (identifier issue).")
        print(f"2. Found {len(final_report_one)} unique Customer ID/Name pairs associated with exactly one email.")
        
    # Sort and return the two DataFrames
    return (
        final_report_morethan_one.sort_values(by='Name').reset_index(drop=True), 
        final_report_one.sort_values(by='Name').reset_index(drop=True)
    )
name_email_issues, name_one = analyze_name_email_correlation(customer_data)

display(name_email_issues.head(10))
display(name_one.head(10))


Analysis Complete: No data quality issues found (All Customer ID/Name pairs have 1 unique email).
2. Found 100000 unique Customer ID/Name pairs associated with exactly one email.


,Customer Id,Name,Emails,Website,unique_email_count,unique_website_count


,Customer Id,Name,Emails,Website,unique_email_count,unique_website_count
0,e77c40dFab505df,aaron aguirre,[leah44@cross-young.com],[http://www.yoder.org/],1,1
1,1D07f142aE2AbCa,aaron allison,[vcannon@jacobs-meyer.com],[http://www.rowland.biz/],1,1
2,Db7df5F61691ccD,aaron barker,[bradleyholland@bowen.com],[http://www.benson.biz/],1,1
3,acc0b2CBf5CeFD5,aaron barnett,[xsherman@nicholson.com],[http://payne-hatfield.com/],1,1
4,A133d2D1abd1F93,aaron barrera,[jasminbridges@barrera.org],[https://moss-howell.com/],1,1
5,4CDDBed7f5EFBe6,aaron bautista,[wwalters@little.info],[https://parker-gilbert.info/],1,1
6,8Ab71CeA4dfa3dB,aaron baxter,[christy87@chase.info],[https://walters-logan.com/],1,1
7,72a1e6d0374765A,aaron beltran,[malikhodge@liu.net],[http://www.stevens.biz/],1,1
8,9a1D42726C3f3F2,aaron benitez,[sfisher@benitez.com],[http://ritter.com/],1,1
9,20DDa14d1e6b42d,aaron bennett,[hayden65@harris.org],[http://www.armstrong.net/],1,1


In [10]:
# Mapping Analysis in the level of Website
def website_customer_mapping(df):
    def get_unique_values_as_list(series: pd.Series) -> List[str]:
        """Convert unique values to a list of strings, handling NaNs."""
        return list(set(series.dropna().astype(str).tolist()))
    
    working_df = df.copy()

    # Required columns validation
    required_cols = ['Website', 'Company', 'First Name', 'Last Name', 'Email']
    for col in required_cols:
        if col not in working_df.columns:
            print(f"Error: Required column '{col}' not found in DataFrame.")
            return pd.DataFrame(), pd.DataFrame()

    # Validate Customer ID column
    customer_id_col = next((col for col in ['Customer_ID', 'Customer Id'] if col in working_df.columns), None)
    if not customer_id_col:
        print("Error: 'Customer_ID' or 'Customer Id' columns required.")
        return pd.DataFrame(), pd.DataFrame()

    print("\n-> Starting Website-Level Mapping Analysis:")
    print("-> Evaluating uniqueness of Company, Customer ID, Email, and Names per Website.")

    # Group by Website
    website_report = working_df.groupby('Website').agg(
        Companies=('Company', get_unique_values_as_list),
        Customer_IDs=(customer_id_col, get_unique_values_as_list),
        Emails=('Email', get_unique_values_as_list),
        Names=('Full Name', get_unique_values_as_list)
    ).reset_index()

    # Count unique attributes
    website_report['unique_company_count'] = website_report['Companies'].apply(len)
    website_report['unique_customer_id_count'] = website_report['Customer_IDs'].apply(len)
    website_report['unique_email_count'] = website_report['Emails'].apply(len)
    website_report['unique_name_count'] = website_report['Names'].apply(len)

    # Identify potential data quality issues
    issue_criteria = (website_report['unique_customer_id_count'] > 1)

    website_multiple_customers = website_report[issue_criteria]
    website_single_customer = website_report[~issue_criteria]

    # Summary output
    if website_multiple_customers.empty:
        print("\nAnalysis Complete: No issues found. Each website belongs to only 1 customer.")
        print(f"   Clean websites: {len(website_single_customer)}")
    else:
        print(f"\nAnalysis Complete:")
        print(f"1. Found {len(website_multiple_customers)} Websites associated with MULTIPLE customers.")
        print(f"2. Found {len(website_single_customer)} Websites associated with EXACTLY 1 customer.")

    return (
        website_multiple_customers.sort_values(by='Website').reset_index(drop=True),
        website_single_customer.sort_values(by='Website').reset_index(drop=True)
    )


# Running the analysis
website_issue_report, website_clean_report = website_customer_mapping(customer_data)
display(website_issue_report.head(10))
display(website_clean_report.head(10))



-> Starting Website-Level Mapping Analysis:
-> Evaluating uniqueness of Company, Customer ID, Email, and Names per Website.

Analysis Complete:
1. Found 12012 Websites associated with MULTIPLE customers.
2. Found 38459 Websites associated with EXACTLY 1 customer.


,Website,Companies,Customer_IDs,Emails,Names,unique_company_count,unique_customer_id_count,unique_email_count,unique_name_count
0,http://abbott.com/,"[crane-hernandez, montes plc, frost-noble, vil...","[AbF7a81BB35ddc7, e6ecbc1d069cb16, 4EE7aD0CfeA...","[zmolina@woodward.info, vkrause@cruz.net, hoto...","[ariel mclaughlin, mercedes harrell, warren mo...",8,8,8,8
1,http://abbott.info/,"[wise and sons, shannon, thomas and haney]","[12c89fA6914Cc94, 0A21e9cd7f5a6C1]","[shawn43@massey-sampson.org, rodney83@bridges....","[ronnie malone, tammy savage]",2,2,2,2
2,http://acevedo.com/,"[valenzuela-cannon, whitney, roy and luna, lee...","[3bab84Aef9b8A6C, baC1d468c8a4d9C, cCDAfA2748B...","[stevenperry@welch.info, mcfarlanddominic@mcbr...","[martin arellano, erik craig, barry walker, hu...",14,14,14,14
3,http://acevedo.org/,"[larson-haas, cortez, chaney and dyer]","[EF6Fe1ea9c2BAdF, ab81bDf6014b2f3]","[annaburgess@frye.com, maxlynch@wilcox.com]","[jackson jennings, mandy nelson]",2,2,2,2
4,http://acosta.biz/,"[callahan, forbes and logan, nixon group]","[ab4e6bd5Aab3ca3, DABDFdFA9dDcED1]","[devonbarry@cain.com, roy94@benitez.com]","[valerie drake, cynthia bernard]",2,2,2,2
5,http://acosta.com/,"[jenkins, combs and king, duran, frye and hine...","[C42AA1AcBcaEa57, a4fe71B313A7bfF, 0ae23E73971...","[susan34@mcgee.com, amanda80@middleton.com, ai...","[stacie choi, pedro ritter, linda mckee, allen...",11,11,11,11
6,http://acosta.info/,"[medina, stephenson and frost, friedman, owens...","[d10Dc0fCfeBcF74, 7F22b70f32a17d0, C57E9E9b9f4...","[staciehaas@lawrence.info, ashleedorsey@randol...","[mercedes rubio, abigail bartlett, monica maye...",5,5,5,5
7,http://acosta.net/,"[gentry, lawrence and daniel, cisneros ltd, fr...","[3a52ac5aeCcD9c2, 5dD5EAFeAea3f9a, AC9032CBfD9...","[leonjade@boyle.com, nboyle@lozano.info, krame...","[misty kirby, ashlee orozco, andres wang, came...",4,4,4,4
8,http://acosta.org/,"[arellano-patton, carrillo group]","[15dFA4170e192D5, 441Fc2Da3A3b8ab]","[codymerritt@robertson.com, wendygiles@austin....","[ross griffin, rodney castro]",2,2,2,2
9,http://adams.biz/,"[cooper-howe, arias plc, burnett-pitts]","[62086f74Fde002f, 3328269aceBaA1C, d2cB799b6Bb...","[ashley24@bush.net, dsilva@curtis.com, savagej...","[felicia cooper, kaitlyn joyce, bernard winters]",3,3,3,3


,Website,Companies,Customer_IDs,Emails,Names,unique_company_count,unique_customer_id_count,unique_email_count,unique_name_count
0,http://abbott-bowers.com/,"[miles, allen and bates]",[0C9e1e80FcDF32A],[roberto72@madden-blake.org],[tristan french],1,1,1,1
1,http://abbott-castillo.info/,[madden-white],[e9BF9d6A7A2F5f3],[triciaray@ross-glass.org],[jon wolf],1,1,1,1
2,http://abbott-collins.com/,[abbott group],[98CDEBEb2F1FFf5],[stewartmeredith@norris.net],[tony oconnell],1,1,1,1
3,http://abbott-cummings.info/,[mcmahon and sons],[461Edbc388A718C],[burkekarina@cline-goodwin.com],[john foley],1,1,1,1
4,http://abbott-english.net/,"[christian, murillo and howell]",[3d21eE9dfCaE49B],[donovangina@snow.com],[levi gregory],1,1,1,1
5,http://abbott-mccoy.com/,[quinn and sons],[5C3F5ceC982dEfF],[veronica31@velazquez.com],[katie spence],1,1,1,1
6,http://abbott-mcpherson.com/,[patrick-hall],[BC51c0b0ffFbD9d],[phayes@boone-hale.com],[damon arnold],1,1,1,1
7,http://abbott-scott.com/,[bradford plc],[D9c3c3115535C4E],[mbarnes@ball-anthony.com],[melody barber],1,1,1,1
8,http://abbott.biz/,"[grant, fleming and berry]",[E878Ea16c5Dba86],[briangibson@benton.com],[edwin gross],1,1,1,1
9,http://abbott.net/,"[robles, knox and kim]",[9f7c414BD5B49F8],[mcdowellsue@welch.com],[roger patton],1,1,1,1


#### Conclusion

Based on the analysis conducted, we confirm that there are no missing (null) values in the provided dataset. Several assumptions were validated to ensure data consistency and integrity:

1. Although no null values are present, hidden or irregular missing values were checked through top-value frequency analysis to ensure completeness.

2. Phone numbers are not the sole unique identifier for customers, as personal and corporate email addresses also serve as unique identifiers. Therefore, no data cleansing or modification is required for the phone number field at this stage.

3. Website, Email, and Full Name demonstrate a unique relationship with Customer ID, indicating strong entity consistency across these fields.

4. A single company may be associated with multiple Customer IDs, which aligns with real-world business relationships (e.g., multiple customers or contacts within the same organization).

Overall, the dataset shows strong integrity and uniqueness across key attributes, allowing us to proceed confidently to deeper exploratory analysis.

## 3. Exploratory Data Analysis

To derive meaningful insights from this subscription dataset, we will follow a structured analytical approach. Since the data represents customer subscriptions, our objective is to understand subscription patterns and behaviors across different identifiers.

This EDA will cover the following key areas with total 2 Phases of Analysis:

1. General Aggregated Overview: Give high level understanding of correlation between every identifier level with the subscription date
2. Monthly Pattern Deep Dive: Going deep to every top pattern that happen in the data that we find.

In [11]:
class SubscriptionPatternAnalyzer:
    """
    Analyzes subscription data in two tiers:
    1. General Aggregated Overview (Top 10 categories, Monthly overall trend).
    2. Deep Dive into Monthly Patterns for Top 3 of each category.
    """
    def __init__(self, df: pd.DataFrame, top_n: int = 10, top_n_deep_dive: int = 3):
        """Initializes the analyzer and prepares data."""
        self.df = df.copy()
        self.TOP_N = top_n
        self.TOP_N_DEEP_DIVE = top_n_deep_dive
        self.top_categories: Dict[str, List[str]] = {}
        
        print(f"Analyzer Initialized. General Focus: Top {self.TOP_N}. Deep Dive Focus: Top {self.TOP_N_DEEP_DIVE}.")
        self._data_prep()

    def _data_prep(self):
        """Prepares the data by ensuring correct date types and identifying top categories."""
        date_col = 'Subscription Date'
        
        try:
            # Standardize column name and convert to datetime
            self.df.rename(columns={'Subscription Date': 'SubscriptionDate'}, inplace=True)
            self.df['SubscriptionDate'] = pd.to_datetime(self.df['SubscriptionDate'], errors='coerce')
            self.df.dropna(subset=['SubscriptionDate', 'Country', 'Website', 'Company', 'City'], inplace=True)
            
            # Create a Monthly column for aggregation
            self.df['YearMonth'] = self.df['SubscriptionDate'].dt.to_period('M')

            # Identify Top N categories for deep dive
            categories = ['Country', 'Website', 'Company', 'City']
            for cat in categories:
                top_values = self.df[cat].value_counts().head(self.TOP_N_DEEP_DIVE).index.tolist()
                self.top_categories[cat] = top_values

            print(f"Data preparation complete: {len(self.df)} valid records remaining.")
            for cat, values in self.top_categories.items():
                print(f"Top {self.TOP_N_DEEP_DIVE} {cat} for deep dive: {', '.join(values)}")

        except KeyError as e:
            print(f"Error during data preparation: A required column is missing: {e}.")
            
    def _display_figure(self, fig: go.Figure, title: str):
        """Helper method to display Plotly figures."""
        print(f"\n--- Displaying Figure: {title} ---")
        fig.show()

    # ----------------------------------------------------
    # Phase 1: General Aggregated Overview (Top 10)
    # ----------------------------------------------------
    def phase_1_general_overview(self):
        """Plots top 10 categories and overall monthly trend."""
        print("\n" + "="*80)
        print(f"Phase 1: General Global Overview (Top {self.TOP_N} Metrics)")
        print("="*80)

        # 1. Overall Monthly Subscription Trend (Smoother View)
        df_monthly_total = self.df.groupby('YearMonth').size().reset_index(name='Total Monthly Subscriptions')
        df_monthly_total['YearMonth'] = df_monthly_total['YearMonth'].astype(str)
        
        fig_monthly_trend = px.line(df_monthly_total, x='YearMonth', y='Total Monthly Subscriptions', 
                                    title='Overall Global Subscription Trend (Monthly Aggregation)',
                                    labels={'Total Monthly Subscriptions': 'Count', 'YearMonth': 'Month'},
                                    template='plotly_dark')
        fig_monthly_trend.update_traces(mode='lines+markers', marker=dict(size=6))
        self._display_figure(fig_monthly_trend, 'Global Monthly Trend')
        


        # 2. Top 10 Bar Charts (Country, Website, Company, City)
        categories = ['Country', 'Website', 'Company', 'City']
        for cat in categories:
            cat_counts = self.df[cat].value_counts().head(self.TOP_N).reset_index()
            cat_counts.columns = [cat, 'Subscription Count']
            
            fig_bar = px.bar(cat_counts, x=cat, y='Subscription Count', 
                             color='Subscription Count', 
                             title=f'Global Top {self.TOP_N} {cat} by Subscription Volume',
                             template='plotly_dark')
            self._display_figure(fig_bar, f'Top {self.TOP_N} {cat} Volume')
            


    # ----------------------------------------------------
    # Phase 2: Monthly Pattern Deep Dive (Top 3)
    # ----------------------------------------------------
    def phase_2_monthly_pattern_deep_dive(self):
        """Plots monthly subscription patterns for the top 3 of each key category."""
        print("\n" + "="*80)
        print(f"Phase 2: Monthly Pattern Deep Dive for Top {self.TOP_N_DEEP_DIVE} Categories")
        print("="*80)
        
        categories = ['Country', 'Website', 'Company', 'City']
        
        for cat in categories:
            top_values = self.top_categories[cat]
            if not top_values:
                print(f"Skipping deep dive for {cat}: No top values found.")
                continue

            print(f"\n--- Analyzing Monthly Trends for Top {self.TOP_N_DEEP_DIVE} {cat} ---")
            
            # Filter the main DataFrame to include only the top N values for this category
            df_filtered = self.df[self.df[cat].isin(top_values)]
            
            # Group by Month and the Category, then count subscriptions
            df_monthly_pattern = df_filtered.groupby(['YearMonth', cat]).size().reset_index(name='Monthly Subscriptions')
            df_monthly_pattern['YearMonth'] = df_monthly_pattern['YearMonth'].astype(str)

            # Create the Line Chart for comparison
            fig_line = px.line(df_monthly_pattern, x='YearMonth', y='Monthly Subscriptions', 
                               color=cat, 
                               title=f'Monthly Subscription Pattern Comparison: Top {self.TOP_N_DEEP_DIVE} {cat}',
                               labels={'Monthly Subscriptions': 'Count', 'YearMonth': 'Month'},
                               template='plotly_dark')
            fig_line.update_traces(mode='lines+markers', marker=dict(size=6))
            self._display_figure(fig_line, f'Monthly Pattern for Top {cat}')
        

    # ----------------------------------------------------
    # Execution
    # ----------------------------------------------------
    def run_all_phases(self):
        """Runs the complete analytical process."""
        self.phase_1_general_overview()
        self.phase_2_monthly_pattern_deep_dive()
        print("\n" + "="*80)
        print("Tiered Subscription Analysis Complete.")
        print("="*80)

In [12]:
analyzer = SubscriptionPatternAnalyzer(customer_data)
analyzer.run_all_phases()

Analyzer Initialized. General Focus: Top 10. Deep Dive Focus: Top 3.
Data preparation complete: 100000 valid records remaining.
Top 3 Country for deep dive: congo, korea, saudi arabia
Top 3 Website for deep dive: https://guzman.com/, http://www.maxwell.com/, https://www.nunez.com/
Top 3 Company for deep dive: wilkerson ltd, campbell ltd, acosta ltd
Top 3 City for deep dive: lake frederick, west alec, east jeremy

Phase 1: General Global Overview (Top 10 Metrics)

--- Displaying Figure: Global Monthly Trend ---



--- Displaying Figure: Top 10 Country Volume ---



--- Displaying Figure: Top 10 Website Volume ---



--- Displaying Figure: Top 10 Company Volume ---



--- Displaying Figure: Top 10 City Volume ---



Phase 2: Monthly Pattern Deep Dive for Top 3 Categories

--- Analyzing Monthly Trends for Top 3 Country ---

--- Displaying Figure: Monthly Pattern for Top Country ---



--- Analyzing Monthly Trends for Top 3 Website ---

--- Displaying Figure: Monthly Pattern for Top Website ---



--- Analyzing Monthly Trends for Top 3 Company ---

--- Displaying Figure: Monthly Pattern for Top Company ---



--- Analyzing Monthly Trends for Top 3 City ---

--- Displaying Figure: Monthly Pattern for Top City ---



Tiered Subscription Analysis Complete.
